In [ ]:
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
from psycopg2 import pool, extras
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import psycopg2

# Configuration: Set to True to process from the bottom (latest years first)
PROCESS_FROM_BOTTOM = True  # Set to False on the other computer

# Database connection parameters
db_params = {
    'dbname': 'StockSelector',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # Update to server IP if running on different machines
    'port': '5432'
}

# Create a threaded connection pool
conn_pool = pool.ThreadedConnectionPool(1, 20, **db_params)

# Precompile the regex pattern for us-gaap tags
us_gaap_regex = re.compile(r'us-gaap:.*', re.I)

# Path to filings folder
filings_folder = r"C:\Users\willi\Documents\GitHub\Stock-Selector\data\edgar\filings"

def get_fiscal_year(period_element):
    date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%d-%b-%Y']
    for tag in ['instant', 'endDate', 'startDate']:
        date_tag = period_element.find(tag)
        if date_tag:
            date_str = date_tag.text.strip()
            for fmt in date_formats:
                try:
                    return datetime.strptime(date_str, fmt).year
                except ValueError:
                    continue
    return None

def process_file(args):
    cik, year_folder, file_path = args
    conn = None
    try:
        conn = conn_pool.getconn()
        with conn.cursor() as cursor:
            # Skip file if already processed
            cursor.execute("SELECT 1 FROM processed_files WHERE file_path = %s", (file_path,))
            if cursor.fetchone():
                return None

            # Read the file
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
                return None

            # Quick check for XBRL content
            if '<context' not in content.lower():
                cursor.execute("INSERT INTO processed_files (file_path) VALUES (%s)", (file_path,))
                conn.commit()
                return None

            soup = BeautifulSoup(content, 'lxml')
            context_elements = soup.find_all('context')
            if not context_elements:
                cursor.execute("INSERT INTO processed_files (file_path) VALUES (%s)", (file_path,))
                conn.commit()
                return None

            # Build a mapping of context id to fiscal year for eligible periods
            context_to_year = {}
            for context in context_elements:
                context_id = context.get('id')
                period = context.find('period')
                if period:
                    fiscal_year = get_fiscal_year(period)
                    if fiscal_year and fiscal_year >= 2009:
                        context_to_year[context_id] = fiscal_year

            if not context_to_year:
                cursor.execute("INSERT INTO processed_files (file_path) VALUES (%s)", (file_path,))
                conn.commit()
                return None

            # Collect fiscal years in the file
            file_fiscal_years = set(context_to_year.values())

            # Check if all fiscal years in the file are already in the database for this CIK
            cursor.execute("SELECT DISTINCT fiscal_year FROM financials WHERE cik = %s", (cik,))
            existing_fy = set(row[0] for row in cursor.fetchall())
            if all(fy in existing_fy for fy in file_fiscal_years):
                cursor.execute("INSERT INTO processed_files (file_path) VALUES (%s)", (file_path,))
                conn.commit()
                return None

            # Extract metrics using the precompiled regex
            metrics = []
            for element in soup.find_all(us_gaap_regex):
                try:
                    value = float(element.text.strip())
                except ValueError:
                    continue
                context_ref = element.get('contextref')
                if context_ref in context_to_year:
                    fiscal_year = context_to_year[context_ref]
                    tag = element.name.lower()
                    metrics.append((cik, fiscal_year, tag, value))

            # Insert metrics if any were found
            if metrics:
                extras.execute_batch(cursor,
                    """
                    INSERT INTO financials (cik, fiscal_year, metric, value)
                    VALUES (%s, %s, %s, %s)
                    ON CONFLICT DO NOTHING
                    """,
                    metrics
                )
            # Mark the file as processed
            cursor.execute("INSERT INTO processed_files (file_path) VALUES (%s)", (file_path,))
            conn.commit()
            return len(metrics)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None
    finally:
        if conn:
            conn_pool.putconn(conn)

def create_tables():
    conn = None
    try:
        conn = conn_pool.getconn()
        with conn.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS financials (
                    cik TEXT,
                    fiscal_year INTEGER,
                    metric TEXT,
                    value NUMERIC,
                    PRIMARY KEY (cik, fiscal_year, metric)
                );
                CREATE TABLE IF NOT EXISTS processed_files (
                    file_path TEXT PRIMARY KEY,
                    processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)
            conn.commit()
    finally:
        if conn:
            conn_pool.putconn(conn)

# Create tables
create_tables()

# Build the list of files to process
file_list = []
for cik_folder in os.listdir(filings_folder):
    cik_path = os.path.join(filings_folder, cik_folder)
    if not os.path.isdir(cik_path):
        continue
    cik = cik_folder
    for year_folder in os.listdir(cik_path):
        try:
            year_int = int(year_folder)
        except ValueError:
            continue
        # Only process filings from 2009 to 2015 (inclusive)
        if year_int < 2009 or year_int > 2015:
            continue
        year_path = os.path.join(cik_path, year_folder)
        if not os.path.isdir(year_path):
            continue
        files = [f for f in os.listdir(year_path) if '10-K' in f]
        if files:
            file_path = os.path.join(year_path, files[0])
            file_list.append((cik, year_folder, file_path))

# Sort the file_list by year_folder and then cik
file_list = sorted(file_list, key=lambda x: (x[1], x[0]))  # x[1] is year_folder, x[0] is cik

if PROCESS_FROM_BOTTOM:
    file_list = file_list[::-1]
    print("Processing from the bottom (latest years first)")
else:
    print("Processing from the top (earliest years first)")

total_files = len(file_list)
processed_count = 0
metrics_inserted = 0

# Process files in parallel using a thread pool
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, args) for args in file_list]
    for future in tqdm(as_completed(futures), total=total_files, desc="Processing 10-K files"):
        result = future.result()
        processed_count += 1
        if result is not None:
            metrics_inserted += result
        if processed_count % 100 == 0:
            print(f"Processed {processed_count}/{total_files} files, inserted {metrics_inserted} metrics")

print(f"Processing complete. Total files processed: {processed_count}, Total metrics inserted: {metrics_inserted}")

Processing 10-K files:   0%|          | 109/46863 [00:01<14:00, 55.65it/s]

Processed 100/46863 files, inserted 0 metrics


Processing 10-K files:   0%|          | 205/46863 [00:04<19:46, 39.34it/s]

Processed 200/46863 files, inserted 0 metrics


Processing 10-K files:   1%|          | 305/46863 [00:06<17:53, 43.35it/s]

...

Processing 10-K files:  14%|█▍        | 6600/46863 [1:04:59<361:49:00, 32.35s/it]

Processed 6600/46863 files, inserted 20158 metrics


Processing 10-K files:  14%|█▍        | 6700/46863 [1:14:23<80:27:57,  7.21s/it] 

Processed 6700/46863 files, inserted 27931 metrics


Processing 10-K files:  14%|█▍        | 6795/46863 [12:39:18<74:37:23,  6.70s/it]     


In [3]:
import pandas as pd
import os
import shutil

# Step 1: Read the CSV file
# Replace the path with your actual CSV file location
csv_path = 'C:/Users/willi/Documents/GitHub/Stock-Selector/data/consolidated_stock_list.csv'
df = pd.read_csv(csv_path, dtype={'cik': str})

# Step 2: Filter rows where ipo_date is "Not Found" and get unique CIKs
ciks_to_delete = df[df['ipo_date'] == "Not Found"]['cik'].unique().tolist()

# Step 3: Define the filings directory
filings_dir = 'C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings'

# Step 4: Process all CIK folders
deleted_cik_count = 0
deleted_year_count = 0

for cik_folder in os.listdir(filings_dir):
    folder_path = os.path.join(filings_dir, cik_folder)
    if not os.path.isdir(folder_path):
        continue

    cik = cik_folder

    # Delete entire CIK folder if it has no IPO date
    if cik in ciks_to_delete:
        try:
            shutil.rmtree(folder_path)
            deleted_cik_count += 1
            print(f"Deleted folder for CIK {cik} (no IPO date)")
        except Exception as e:
            print(f"Error deleting folder for CIK {cik}: {e}")
        continue  # Move to the next CIK folder

    # For CIKs with an IPO date, check year folders
    for year_folder in os.listdir(folder_path):
        year_path = os.path.join(folder_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        try:
            year = int(year_folder)
            if year < 2010:
                shutil.rmtree(year_path)
                deleted_year_count += 1
                print(f"Deleted year folder {year_path} (year < 2010)")
        except ValueError:
            print(f"Skipping non-numeric year folder: {year_folder} in CIK {cik}")
        except Exception as e:
            print(f"Error deleting year folder {year_path}: {e}")

# Step 5: Summary
print(f"Deletion complete. Deleted {deleted_cik_count} CIK folders and {deleted_year_count} year folders.")

Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000045\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000180\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000209\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000228\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000229\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000230\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000232\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000234\2009 (year < 2010)
Deleted year folder C:/Users/willi/Documents/GitHub/Stock-Selector/data/edgar/filings\1000278\20